In [18]:
import numpy as np
import math
import random
from scipy.stats import poisson
import time

## Functions

In [19]:
I = 8  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

In [20]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [21]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [22]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probSize = calcExponentialLimit(beta*N)+1
calculate_p(beta, probSize, precision)


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [23]:
def calculate_v(p, beta, precision, precision_limit, n, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*n) + 1
  v = np.zeros((n+1, limit+d))
  add_v = np.zeros((n+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, n+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(n+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probSize, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.24794646e-47, 1.34778218e-45, 7.27802377e-44, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.54008828e-51, 1.80190329e-49, 1.05411343e-47, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.38695255e-07, 2.54327543e-07, 1.13518871e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 170)

In [24]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########



v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
calculateProbabilities(x, precision, limit, v, I, d)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.15690521e-01, 9.10903190e-02, 1.17116124e-01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.75360063e-11, 9.39355084e-11, 4.97199148e-10, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.62371117e-12, 6.59128290e-12, 2.77562616e-11, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.34072091e-12, 6.30424881e-12, 2.07008832e-11, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.83216691e-22, 2.85994871e-21, 1.67197250e-20, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 170)

In [25]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, n, no_show, beta):
  return (I * d) + tardiness - (n * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, n):
  w = np.zeros((I+1, n+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= n
  return waitingtime

In [26]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

In [27]:
def calcFitness(x, beta, precision, limit, v, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, n, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, n)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probSize, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)


Schedule: [1 4 0 1 0 3 1 4],
Objective value: 49.813140724994156,
Prob calculation time: 0.111767 sec,
Waiting time (timer): 38.54834317685784 (0.159089 sec),
Idle time (timer): -0.01032757624830083 (0.000003 sec),
Tardiness (timer): 85.9896724237517 (0.000082 sec)



<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [28]:
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    norm1 = sum(child1)
    p1 = child1 / norm1
    
    # The number of swaps should not exceed the maximum number of patients in any interval. Otherwise it might happen that the number of patients that are transfered exceeds availability, resulting in a negative amount.
    max1 = int(max(child1))
    max2 = int(max(child2))
    n = min(nSwaps, max1, max2)
    
    # Swap patients parent1 to parent 2
    i = np.random.choice(I, nSwaps, p=p1)
    for i in i:
        child1[i] += -1
        child2[i] += 1
    
    # Swap patients parent2 to parent 1
    norm2 = sum(child2)
    p2 = child2 / norm2
    j = np.random.choice(I, nSwaps, p=p2)
    for j in j:
        child1[j] += 1
        child2[j] += -1
    return(child1, child2)

##########
## TEST ##
##########


parent1 = np.random.choice(5, I)
parent2 = np.random.choice(5, I)

print(parent1, parent2)
n = 3
children = crossover(parent1, parent2, n)
children


[1 3 3 1 2 1 1 2] [1 1 0 4 0 4 3 0]


(array([1, 4, 3, 1, 2, 1, 0, 2]), array([1, 0, 0, 4, 0, 4, 4, 0]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [29]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[1 4 3 1 2 1 0 2]
[1 0 3 1 2 1 4 2]


## Classes

Create a data object to store and generate initial data.

In [42]:
class Data:
    def __init__(self, I=40, N=8, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=None, alpha_T=None, alpha_W=None):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self_p, self.precision_limit, self.v, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'p', 'precision_limit', 'v', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData['popSize']
probData = newDataObject.v
probData



array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.24794646e-47, 1.34778218e-45, 7.27802377e-44, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.54008828e-51, 1.80190329e-49, 1.05411343e-47, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.38695255e-07, 2.54327543e-07, 1.13518871e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [ ]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)

##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
schedule = scheduleObject.get_schedule()
schedule

[0 0 0 0 0 0 0 0]


array([2, 0, 2, 4, 4, 1, 1, 1])

Create population of individuals

In [ ]:
class Population:
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.popSize = self.dataObject.get_initial_data()['popSize']
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals()    

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.popSize):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    
    
##########
## TEST ##
##########

populationObject = Population(dataObject)
populationObject.individuals


[array([3, 1, 1, 2, 2, 3, 0, 3]),
 array([3, 1, 2, 2, 2, 2, 1, 2]),
 array([3, 2, 3, 1, 3, 1, 1, 1])]

## Fitness function

In [ ]:
##########
## TEST ##
##########

schedules = populationObject.individuals

fitnessScores = list(map(lambda x: calcFitness(x, beta, precision, limit, v, N, no_show,
                                                I, d, eind, alpha_W, alpha_I, alpha_T)['objVal'], schedules))
fitnessScores


Schedule: [3 1 1 2 2 3 0 3],
Objective value: 56.053169682582705,
Prob calculation time: 0.122942 sec,
Waiting time (timer): 45.29757640830109 (0.211448 sec),
Idle time (timer): -0.10976813456289847 (0.000003 sec),
Tardiness (timer): 94.89023186543712 (0.000084 sec)

Schedule: [3 1 2 2 2 2 1 2],
Objective value: 56.586208985663134,
Prob calculation time: 0.098981 sec,
Waiting time (timer): 46.6301747804201 (0.212458 sec),
Idle time (timer): -0.10976821084150856 (0.000002 sec),
Tardiness (timer): 94.89023178915849 (0.000068 sec)

Schedule: [3 2 3 1 3 1 1 1],
Objective value: 57.65224310741141,
Prob calculation time: 0.119201 sec,
Waiting time (timer): 49.29526024125805 (0.171547 sec),
Idle time (timer): -0.1097683151530191 (0.000003 sec),
Tardiness (timer): 94.89023168484697 (0.000074 sec)



[56.053169682582705, 56.586208985663134, 57.65224310741141]